# ZAlign

## 生产添加 bias 的输入JSON

In [1]:
from zalign import ZScore
from rdkit import Chem
import json
import os
import numpy as np
def show_similarity_matrix(matrix):
    # 用plotly绘制相似性得分矩阵的热力图，可以交互式查看横纵坐标
    import plotly.graph_objs as go
    import numpy as np

    # 构造x轴和y轴标签（原子索引从1开始）
    x_labels = [str(i+1) for i in range(matrix.shape[1])]
    y_labels = [str(i+1) for i in range(matrix.shape[0])]

    heatmap = go.Heatmap(
        z=matrix,
        x=x_labels,
        y=y_labels,
        colorscale='Hot',
        colorbar=dict(title='Score'),
        hoverongaps=False,
        hovertemplate='Query Atom: %{y}<br>Template Atom: %{x}<br>Score: %{z:.1f}<extra></extra>'
    )

    layout = go.Layout(
        title='Similarity Score Matrix',
        xaxis=dict(title='Template Molecule Atom Index'),
        yaxis=dict(title='Query Molecule Atom Index'),
        width=900,
        height=900
    )

    fig = go.Figure(data=[heatmap], layout=layout)
    fig.show()


def generate_zalign_input(fp_sdf_q, fp_sdf_t, fp_json_ori, fp_json_align):
    mol_q = Chem.SDMolSupplier(fp_sdf_q, removeHs=False)[0]
    mol_t = Chem.SDMolSupplier(fp_sdf_t, removeHs=False)[0]

    matrix = ZScore(mol_q, mol_t).score()

    #------- 准备输入JSON
    with open(fp_json_ori, "r") as f:
        data = json.load(f)

    k = list(set(data.keys()) - {"score", "receptor"})[0]

    # 得到所有原子的坐标
    coords_t = mol_t.GetConformer().GetPositions().tolist()
    coords_q = mol_q.GetConformer().GetPositions().tolist()
    center_t = np.mean(coords_t, axis=0)
    center_q = np.mean(coords_q, axis=0)
    translation = center_t - center_q
    
    natom_t = mol_t.GetNumAtoms()
    natom_q = mol_q.GetNumAtoms()

    for i in range(natom_q):

        biases = []
        for j in range(natom_t): # each template atom
            biases.append([*coords_t[j], float(matrix[i][j]), 0.])
        
        # add biases to this
        data[k]["atoms"][i].append(biases)

        # Align two centers
        data[k]["atoms"][i][0] += translation[0]
        data[k]["atoms"][i][1] += translation[1]
        data[k]["atoms"][i][2] += translation[2]

    data["score"] = ["vina", "gaff2"] # `unidock2`-dumped json losts this key 
    with open(fp_json_align, "w") as f:
        json.dump(data, f)
    
    return mol_q, mol_t, matrix

In [ ]:
# cmet
# mol_q, mol_t, matrix = generate_zalign_input(
#     fp_sdf_q="/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_HNE/HNE_Ref.sdf",
#     fp_sdf_t="/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_HNE/HNE_29.sdf",
#     fp_json_ori="/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_HNE/tmp/docking_lccdp-S1_30087_20250825_190851_196850_omk3fehp/ud2_engine_inputs.json",
#     fp_json_align="/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_HNE/ud2_align.json"
# )


In [ ]:
# dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/"
# fp = os.path.join(dp, "tmp", os.listdir(os.path.join(dp, "tmp"))[0], "ud2_engine_inputs.json")
# fp_align = os.path.join(dp, "ud2_align.json")

# fp_sdf_q = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/dock_with_bias/CAT-13g_rand.sdf"
# fp_sdf_t = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/dock_with_bias/CAT-24.sdf"
# fp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/dock_with_bias/ud2_input.json"
# fp_align = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/dock_with_bias/ud2_align.json"

# generate_zalign_input(fp_sdf_q, fp_sdf_t, fp_json_ori, fp_json_align)


In [ ]:
# fp_sdf_q = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace_Chiral/0.sdf"
# fp_sdf_t = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace_Chiral/1.sdf"
# dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace_Chiral/tmp/"
# fp_json_ori = os.path.join(dp, sorted(os.listdir(dp))[-1], "ud2_engine_inputs.json")
# fp_json_align = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace_Chiral/ud2_align.json"

# mol_q, mol_t, matrix = generate_zalign_input(fp_sdf_q, fp_sdf_t, fp_json_ori, fp_json_align)


In [36]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_4IEH_diff_ring/"
fp_sdf_q = os.path.join(dp, "CHEMBL376408_0.6_nM_3d_dock.sdf")
fp_sdf_t = os.path.join(dp, "CHEMBL2322026_8.0_nM_3d_dock.sdf")
dp_tmp = os.path.join(dp, "tmp/")
fp_json_ori = os.path.join(dp_tmp, sorted(os.listdir(dp_tmp))[-1], "ud2_engine_inputs.json")
fp_json_align = os.path.join(dp, "ud2_align.json")

mol_q, mol_t, matrix = generate_zalign_input(fp_sdf_q, fp_sdf_t, fp_json_ori, fp_json_align)

In [35]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_4IEH_ref/"
fp_sdf_q = os.path.join(dp, "CHEMBL376408_prepared.sdf")
fp_sdf_t = os.path.join(dp, "1E9_ref.sdf")
dp_tmp = os.path.join(dp, "tmp/")
fp_json_ori = os.path.join(dp_tmp, sorted(os.listdir(dp_tmp))[-1], "ud2_engine_inputs.json")
fp_json_align = os.path.join(dp, "ud2_align.json")

mol_q, mol_t, matrix = generate_zalign_input(fp_sdf_q, fp_sdf_t, fp_json_ori, fp_json_align)

In [ ]:
mol_q = Chem.SDMolSupplier(fp_sdf_q, removeHs=False)[0]
mol_t = Chem.SDMolSupplier(fp_sdf_t, removeHs=False)[0]
print(mol_q.GetNumAtoms())
print(mol_t.GetNumAtoms())

## 检查每个原子上的bias reward

In [ ]:
a_5 = """  [Total] Bias on 0 is 434.837860
  [Total] Bias on 1 is 437.883270
  [Total] Bias on 2 is 454.205658
  [Total] Bias on 3 is 513.717163
  [Total] Bias on 4 is 433.839905
  [Total] Bias on 5 is 687.286499
  [Total] Bias on 6 is 694.015259
  [Total] Bias on 7 is 497.401917
  [Total] Bias on 8 is 492.694153
  [Total] Bias on 9 is 477.865295
  [Total] Bias on 10 is 466.190033
  [Total] Bias on 11 is 510.029724
  [Total] Bias on 12 is 390.892303
  [Total] Bias on 13 is 407.088562
  [Total] Bias on 14 is 406.983887
  [Total] Bias on 15 is 440.428406
  [Total] Bias on 16 is 768.050476
  [Total] Bias on 17 is 383.888672
  [Total] Bias on 18 is 502.142487
  [Total] Bias on 19 is 454.030273
  [Total] Bias on 20 is 467.322479
  [Total] Bias on 21 is 529.877869
  [Total] Bias on 22 is 256.904388
  [Total] Bias on 23 is 539.857300
  [Total] Bias on 24 is 523.212036
  [Total] Bias on 25 is 276.073395
  [Total] Bias on 26 is 668.550659
  [Total] Bias on 27 is 516.761353
  [Total] Bias on 28 is 446.399628
  [Total] Bias on 29 is 437.192841
  [Total] Bias on 30 is 438.738556
  [Total] Bias on 31 is 447.144531
  [Total] Bias on 32 is 495.780396
  [Total] Bias on 33 is 487.285248
  [Total] Bias on 34 is 447.613831
  [Total] Bias on 35 is 459.363373
  [Total] Bias on 36 is 530.082092
  [Total] Bias on 37 is 452.512909
  [Total] Bias on 38 is 459.059784
  [Total] Bias on 39 is 447.555084
  [Total] Bias on 40 is 523.844543
  [Total] Bias on 41 is 540.435608
  [Total] Bias on 42 is 469.693268
  [Total] Bias on 43 is 455.076965
  [Total] Bias on 44 is 530.074341
  [Total] Bias on 45 is 337.214386
  [Total] Bias on 46 is 693.269165
  [Total] Bias on 47 is 415.706390
  [Total] Bias on 48 is 412.766052
  [Total] Bias on 49 is 682.466980
  [Total] Bias on 50 is 686.112366
  [Total] Bias on 51 is 428.412537
  [Total] Bias on 52 is 694.880859
  [Total] Bias on 53 is 689.878052
  [Total] Bias on 54 is 452.058167
  [Total] Bias on 55 is 436.113495
  [Total] Bias on 56 is 2.341205
  [Total] Bias on 57 is 2.344645
  [Total] Bias on 58 is 2.339919
  [Total] Bias on 59 is 2.310529
  [Total] Bias on 60 is 2.328649
  [Total] Bias on 61 is 2.321392
  [Total] Bias on 62 is 1.997979
  [Total] Bias on 63 is 2.337433
  [Total] Bias on 64 is 2.330834
  [Total] Bias on 65 is 2.367126
  [Total] Bias on 66 is 2.332510
  [Total] Bias on 67 is 2.331291
  [Total] Bias on 68 is 2.330812
  [Total] Bias on 69 is 2.266410
  [Total] Bias on 70 is 2.266515
  [Total] Bias on 71 is 1.672162
  [Total] Bias on 72 is 2.243036
  [Total] Bias on 73 is 2.252204
  [Total] Bias on 74 is 2.335458
  [Total] Bias on 75 is 2.337729
  [Total] Bias on 76 is 2.338017
  [Total] Bias on 77 is 2.335671
  [Total] Bias on 78 is 2.334428
  [Total] Bias on 79 is 2.333882
  [Total] Bias on 80 is 2.332246
  [Total] Bias on 81 is 2.335143
  [Total] Bias on 82 is 2.274140
  [Total] Bias on 83 is 2.261748
  [Total] Bias on 84 is 2.329131
  [Total] Bias on 85 is 2.334845
  [Total] Bias on 86 is 2.331738
  [Total] Bias on 87 is 2.372009
  [Total] Bias on 88 is 2.324117
  [Total] Bias on 89 is 2.307616
  [Total] Bias on 90 is 2.315824
  [Total] Bias on 91 is 2.264395
  [Total] Bias on 92 is 2.273753
  [Total] Bias on 93 is 1.681142
  [Total] Bias on 94 is 2.296904
  [Total] Bias on 95 is 2.276802
  [Total] Bias on 96 is 2.266453
  [Total] Bias on 97 is 2.267425
  [Total] Bias on 98 is 2.280829
  [Total] Bias on 99 is 2.268176
  [Total] Bias on 100 is 2.337898""".strip()

In [ ]:
a_1 = """  [Total] Bias on 0 is 439.055939
  [Total] Bias on 1 is 438.695770
  [Total] Bias on 2 is 451.660797
  [Total] Bias on 3 is 511.726044
  [Total] Bias on 4 is 432.177582
  [Total] Bias on 5 is 700.866211
  [Total] Bias on 6 is 694.197205
  [Total] Bias on 7 is 502.762909
  [Total] Bias on 8 is 494.992096
  [Total] Bias on 9 is 478.340118
  [Total] Bias on 10 is 465.944946
  [Total] Bias on 11 is 509.305664
  [Total] Bias on 12 is 391.718323
  [Total] Bias on 13 is 407.479797
  [Total] Bias on 14 is 406.717682
  [Total] Bias on 15 is 441.354950
  [Total] Bias on 16 is 767.982910
  [Total] Bias on 17 is 383.859344
  [Total] Bias on 18 is 502.184357
  [Total] Bias on 19 is 454.782349
  [Total] Bias on 20 is 467.561157
  [Total] Bias on 21 is 529.477356
  [Total] Bias on 22 is 256.794281
  [Total] Bias on 23 is 540.043335
  [Total] Bias on 24 is 523.219788
  [Total] Bias on 25 is 275.408325
  [Total] Bias on 26 is 667.262268
  [Total] Bias on 27 is 517.092224
  [Total] Bias on 28 is 446.564575
  [Total] Bias on 29 is 435.991425
  [Total] Bias on 30 is 437.471954
  [Total] Bias on 31 is 447.133209
  [Total] Bias on 32 is 495.868958
  [Total] Bias on 33 is 486.947723
  [Total] Bias on 34 is 446.884644
  [Total] Bias on 35 is 459.721191
  [Total] Bias on 36 is 531.010498
  [Total] Bias on 37 is 452.461090
  [Total] Bias on 38 is 459.761566
  [Total] Bias on 39 is 446.889862
  [Total] Bias on 40 is 523.346558
  [Total] Bias on 41 is 540.599243
  [Total] Bias on 42 is 469.026886
  [Total] Bias on 43 is 455.063446
  [Total] Bias on 44 is 529.133667
  [Total] Bias on 45 is 333.590485
  [Total] Bias on 46 is 689.104004
  [Total] Bias on 47 is 401.843262
  [Total] Bias on 48 is 406.287476
  [Total] Bias on 49 is 683.739014
  [Total] Bias on 50 is 686.934021
  [Total] Bias on 51 is 424.693146
  [Total] Bias on 52 is 689.569641
  [Total] Bias on 53 is 690.755981
  [Total] Bias on 54 is 452.370819
  [Total] Bias on 55 is 439.349518
  [Total] Bias on 56 is 2.340793
  [Total] Bias on 57 is 2.341590
  [Total] Bias on 58 is 2.340202
  [Total] Bias on 59 is 2.356138
  [Total] Bias on 60 is 2.353007
  [Total] Bias on 61 is 2.316595
  [Total] Bias on 62 is 2.032474
  [Total] Bias on 63 is 2.339180
  [Total] Bias on 64 is 2.334437
  [Total] Bias on 65 is 2.367949
  [Total] Bias on 66 is 2.332810
  [Total] Bias on 67 is 2.328818
  [Total] Bias on 68 is 2.334094
  [Total] Bias on 69 is 2.268683
  [Total] Bias on 70 is 2.264889
  [Total] Bias on 71 is 1.671653
  [Total] Bias on 72 is 2.246397
  [Total] Bias on 73 is 2.247418
  [Total] Bias on 74 is 2.333455
  [Total] Bias on 75 is 2.335336
  [Total] Bias on 76 is 2.336870
  [Total] Bias on 77 is 2.335640
  [Total] Bias on 78 is 2.333032
  [Total] Bias on 79 is 2.333005
  [Total] Bias on 80 is 2.333049
  [Total] Bias on 81 is 2.333217
  [Total] Bias on 82 is 2.273136
  [Total] Bias on 83 is 2.261105
  [Total] Bias on 84 is 2.327353
  [Total] Bias on 85 is 2.337104
  [Total] Bias on 86 is 2.329123
  [Total] Bias on 87 is 2.372011
  [Total] Bias on 88 is 2.325926
  [Total] Bias on 89 is 2.338611
  [Total] Bias on 90 is 2.315101
  [Total] Bias on 91 is 2.278872
  [Total] Bias on 92 is 2.258702
  [Total] Bias on 93 is 1.664706
  [Total] Bias on 94 is 2.272196
  [Total] Bias on 95 is 2.272880
  [Total] Bias on 96 is 2.264847
  [Total] Bias on 97 is 2.275809
  [Total] Bias on 98 is 2.267298
  [Total] Bias on 99 is 2.285192
  [Total] Bias on 100 is 2.337141""".strip()

In [ ]:
bias_1 = np.array([float(i.strip().split(" ")[-1]) for i in a_1.split("\n")])
bias_5 = np.array([float(i.strip().split(" ")[-1]) for i in a_5.split("\n")])

In [ ]:
arr = np.round(bias_1 - bias_5, 0)
for i in range(0, len(arr), 10):
    print(arr[i:i+10]) 
# 正值说明1更差，负值说明5更差

[ 4.  1. -3. -2. -2. 14.  0.  5.  2.  0.]
[-0. -1.  1.  0. -0.  1. -0. -0.  0.  1.]
[ 0. -0. -0.  0.  0. -1. -1.  0.  0. -1.]
[-1. -0.  0. -0. -1.  0.  1. -0.  1. -1.]
[ -0.   0.  -1.  -0.  -1.  -4.  -4. -14.  -6.   1.]
[ 1. -4. -5.  1.  0.  3. -0. -0.  0.  0.]
[ 0. -0.  0.  0.  0.  0.  0. -0.  0.  0.]
[-0. -0.  0. -0. -0. -0. -0. -0. -0. -0.]
[ 0. -0. -0. -0. -0.  0. -0.  0.  0.  0.]
[-0.  0. -0. -0. -0. -0. -0.  0. -0.  0.]
[-0.]


In [ ]:
show_similarity_matrix(matrix)

## 自动执行 align docking 的流程

In [ ]:
import subprocess as sp
import yaml
import os
import time

# ------------- Configurations
fp_ud2 = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/cmake-build-release/bin/ud2"
fp_json2sdf = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/scripts/json2sdf.py"
dp_config = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_template"
with open(os.path.join(dp_config, "unidock2_free.yaml"), "r") as f: 
    config_free = yaml.safe_load(f)

with open(os.path.join(dp_config, "unidock2_mcs.yaml"), "r") as f:
    config_mcs = yaml.safe_load(f)

with open(os.path.join(dp_config, "ud2.yaml"), "r") as f:
    config_ud2 = yaml.safe_load(f)



def generate_rand(dp:str, fn_pdb:str, fn_sdf_q:str, center:list):

    time_start = time.time()

    # ------------- 执行 unidock2 MCS docking
    fp_config_unidock2 = os.path.join(dp, "unidock2_rand.yaml")
    with open(os.path.join(dp_config, "unidock2_free.yaml"), "r") as f:     
        config_rand = yaml.safe_load(f)

    config_rand["Advanced"]["exhaustiveness"] = 10
    config_rand["Advanced"]["mc_steps"] = 1
    config_rand["Advanced"]["num_pose"] = 1
    config_rand["Advanced"]["opt_steps"] = 1
    config_rand["Preprocessing"]["output_docking_pose_sdf_file_name"] = fn_sdf_q[:-4] + "_rand.sdf"
    config_rand["Settings"]["task"] = "screen"

    with open(fp_config_unidock2, "w") as f:
        yaml.dump(config_rand, f)

    sp.run(f"unidock2 docking -r {fn_pdb} -l {fn_sdf_q} -cf {fp_config_unidock2} -c {center[0]} {center[1]} {center[2]}", shell=True, cwd=dp)


    time_end = time.time()
    print(f"\n\nGenerate random pose time: {time_end - time_start} seconds\n\n")
    return mol_q, mol_t, matrix


def perform_align_docking(dp:str, fn_pdb:str, fn_sdf_q:str, fn_sdf_t:str, center:list, 
    exhaustiveness:int=512, bias_k:float=0.3):

    time_start = time.time()

    # ------------- 执行 unidock2 MCS docking
    fp_config_unidock2 = os.path.join(dp, "unidock2_mcs.yaml")
    config_mcs["Preprocessing"]["reference_sdf_file_name"] = fn_sdf_t
    with open(fp_config_unidock2, "w") as f:
        yaml.dump(config_mcs, f)

    sp.run(f"unidock2 docking -r {fn_pdb} -l {fn_sdf_q} -cf {fp_config_unidock2} -c {center[0]} {center[1]} {center[2]}", shell=True, cwd=dp)


    # ------------- 执行 unidock2, 生产 JSON 输入文件
    dp_tmp = os.path.join(dp, "tmp/")
    os.makedirs(dp_tmp, exist_ok=True)
    fp_config_unidock2 = os.path.join(dp, "unidock2_free.yaml")
    config_free["Preprocessing"]["temp_dir_name"] = "./tmp"
    with open(fp_config_unidock2, "w") as f:
        yaml.dump(config_free, f)

    sp.run(f"unidock2 docking -r {fn_pdb} -l {fn_sdf_q} -cf {fp_config_unidock2} -c {center[0]} {center[1]} {center[2]}", shell=True, cwd=dp)


    # ------------- 加工输入 JSON，添加bias
    fp_sdf_q = os.path.join(dp, fn_sdf_q)
    fp_sdf_t = os.path.join(dp, fn_sdf_t)
    fp_json_ori = os.path.join(dp_tmp, sorted(os.listdir(dp_tmp))[-1], "ud2_engine_inputs.json")
    fp_json_align = os.path.join(dp, "ud2_align.json")

    mol_q, mol_t, matrix = generate_zalign_input(fp_sdf_q, fp_sdf_t, fp_json_ori, fp_json_align)

    # ------------- 执行 ud2, 生产json输出文件
    fp_config_ud2 = os.path.join(dp, "ud2.yaml")
    config_ud2["Settings"]["center_x"] = center[0]
    config_ud2["Settings"]["center_y"] = center[1]
    config_ud2["Settings"]["center_z"] = center[2]
    config_ud2["Advanced"]["bias_k"] = bias_k
    config_ud2["Advanced"]["exhaustiveness"] = exhaustiveness
    with open(fp_config_ud2, "w") as f:
        yaml.dump(config_ud2, f)
    sp.run(f"{fp_ud2} {fp_config_ud2}", shell=True, cwd=dp)

    # ------------- 将输出 JSON 文件转换为 SDF 文件
    dp_res_ud2 = os.path.join(dp, "res2")
    fp_res_ud2 = os.path.join(dp_res_ud2, os.listdir(dp_res_ud2)[0])

    sp.run(f"python {fp_json2sdf} {fp_res_ud2} {fp_sdf_q} ud2_out_bias.sdf", shell=True, cwd=dp)

    time_end = time.time()
    print(f"\n\nAlign docking time: {time_end - time_start} seconds\n\n")
    return mol_q, mol_t, matrix

### 4IEH

In [ ]:
# ------------- User Input
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_4IEH/diff_ring/"
fn_pdb = "4IEH.pdb"
fn_sdf_q = "CHEMBL376408_0.6_nM_3d_dock.sdf"
fn_sdf_t = "CHEMBL2322026_8.0_nM_3d_dock.sdf"
center = [12, 25, 11]

mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 2048, 0.3)


In [11]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_4IEH/same_ring/"
fn_pdb = "4IEH.pdb"
fn_sdf_q = "CHEMBL376408_prepared.sdf"
fn_sdf_t = "CHEMBL2322026_8.0_nM_3d_dock.sdf"
center = [12, 25, 11]

mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 2048, 0.3)



    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit


{'receptor': 'xx.pdb', 'ligand': 'xx.sdf', 'ligand_batch': None, 'center': [12.0, 25.0, 11.0], 'exhaustiveness': 512, 'randomize': True, 'mc_steps': 40, 'opt_steps': -1, 'refine_steps': 5, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': True, 'reference_sdf_file_name': 'CHEMBL2322026_8.0_nM_3d_dock.sdf', 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': '/tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': 'unidock2_out_m

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-28 16:21:45.315] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 16:21:45.315] [info] Receptor loaded: 1101 atoms in box
[2025-08-28 16:21:45.316] [info] Ligands loaded. Total count: 1
[2025-08-28 16:21:45.379] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 16:21:45.379] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-2.7356606 Angstrom, x_hi=27.26434 Angstrom, y_lo=10.112438 Angstrom, y_hi=40.11244 Angstrom, z_lo=-3.686613 Angstrom, z_hi=26.313387 Angstrom, 
constraint_docking=true, exhaustiveness=512, 
mc_steps=40, opt_steps=-1, refine_steps=5, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-28 16:21:45.379] [info] Group small size: 0
[2025-08-28 16:21:45.379] [info] Group medium size: 0
[2025-08-28 16:21:45.379] [info] Group large size: 1
[2025-08-28 16:21:45.379] [info] Group extra size: 0
[2025-08-28 16:21:45.379] [info] Group overflow size: 0
[2025-08-28 16:21:45.379] [info] @@@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:442: UserWarning: PDB file contained CONECT record to TER entry. These are not included in bonds.
  warnings.warn(
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-28 16:21:59.039] [info] ----------------------- RUN Only Scoring -----------------------
[2025-08-28 16:21:59.039] [info] Receptor loaded: 1100 atoms in box
[2025-08-28 16:21:59.040] [info] Ligands loaded. Total count: 1
[2025-08-28 16:21:59.102] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 16:21:59.102] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=-3 Angstrom, x_hi=27 Angstrom, y_lo=10 Angstrom, y_hi=40 Angstrom, z_lo=-4 Angstrom, z_hi=26 Angstrom, 
constraint_docking=false, exhaustiveness=1, 
mc_steps=0, opt_steps=0, refine_steps=0, 
num_pose=1, rmsd_limit=999 Angstrom, 

[2025-08-28 16:21:59.102] [info] Group small size: 0
[2025-08-28 16:21:59.102] [info] Group medium size: 0
[2025-08-28 16:21:59.102] [info] Group large size: 1
[2025-08-28 16:21:59.102] [info] Group extra size: 0
[2025-08-28 16:21:59.102] [info] Group overflow size: 0
[2025-08-28 16:21:59.102] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@@

In [ ]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_4IEH/diff_ring/"
fn_pdb = "4IEH.pdb"
fn_sdf_q = "CHEMBL376408_0.6_nM_3d_dock.sdf"
fn_sdf_t = "CHEMBL2322026_8.0_nM_3d_dock.sdf"
center = [12, 25, 11]

mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 2048, 0.3)

### Bace

In [12]:
dp_Bace = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace_Chiral"
fn_pdb = "Bace.pdb"
fn_sdf_q = "0.sdf"
fn_sdf_t = "1.sdf"
center = [15, -1, -1]

for i in [0, 2, 3, 4]:
    fn_sdf_q = f"{i}.sdf"
    dp = os.path.join(dp_Bace, f"1_{i}")
    mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 512, 0.3)



    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit


{'receptor': 'xx.pdb', 'ligand': 'xx.sdf', 'ligand_batch': None, 'center': [12.0, 25.0, 11.0], 'exhaustiveness': 512, 'randomize': True, 'mc_steps': 40, 'opt_steps': -1, 'refine_steps': 5, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': True, 'reference_sdf_file_name': '1.sdf', 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': '/tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': 'unidock2_out_mcs.sdf'}
Target Center for 

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


[2025-08-28 16:27:26.084] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 16:27:26.085] [info] Receptor loaded: 2390 atoms in box
[2025-08-28 16:27:26.085] [info] Ligands loaded. Total count: 1
[2025-08-28 16:27:26.144] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 16:27:26.144] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=0.0044555664 Angstrom, x_hi=30.004456 Angstrom, y_lo=-16.513166 Angstrom, y_hi=13.486834 Angstrom, z_lo=-15.813394 Angstrom, z_hi=14.186606 Angstrom, 
constraint_docking=true, exhaustiveness=512, 
mc_steps=40, opt_steps=-1, refine_steps=5, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-28 16:27:26.144] [info] Group small size: 0
[2025-08-28 16:27:26.144] [info] Group medium size: 1
[2025-08-28 16:27:26.144] [info] Group large size: 0
[2025-08-28 16:27:26.144] [info] Group extra size: 0
[2025-08-28 16:27:26.144] [info] Group overflow size: 0
[2025-08-28 16:27:26.144] [info] @@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


[2025-08-28 16:27:56.895] [info] ----------------------- RUN Only Scoring -----------------------
[2025-08-28 16:27:56.896] [info] Receptor loaded: 2393 atoms in box
[2025-08-28 16:27:56.896] [info] Ligands loaded. Total count: 1
[2025-08-28 16:27:56.956] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 16:27:56.956] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=0 Angstrom, x_hi=30 Angstrom, y_lo=-16 Angstrom, y_hi=14 Angstrom, z_lo=-16 Angstrom, z_hi=14 Angstrom, 
constraint_docking=false, exhaustiveness=1, 
mc_steps=0, opt_steps=0, refine_steps=0, 
num_pose=1, rmsd_limit=999 Angstrom, 

[2025-08-28 16:27:56.956] [info] Group small size: 0
[2025-08-28 16:27:56.956] [info] Group medium size: 1
[2025-08-28 16:27:56.956] [info] Group large size: 0
[2025-08-28 16:27:56.956] [info] Group extra size: 0
[2025-08-28 16:27:56.956] [info] Group overflow size: 0
[2025-08-28 16:27:56.956] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@

...

[2025-08-28 16:30:17.263] [info] ----------------------- RUN Only Scoring -----------------------
[2025-08-28 16:30:17.263] [info] Receptor loaded: 2393 atoms in box
[2025-08-28 16:30:17.264] [info] Ligands loaded. Total count: 1
[2025-08-28 16:30:17.329] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 16:30:17.329] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=0 Angstrom, x_hi=30 Angstrom, y_lo=-16 Angstrom, y_hi=14 Angstrom, z_lo=-16 Angstrom, z_hi=14 Angstrom, 
constraint_docking=false, exhaustiveness=1, 
mc_steps=0, opt_steps=0, refine_steps=0, 
num_pose=1, rmsd_limit=999 Angstrom, 

[2025-08-28 16:30:17.329] [info] Group small size: 0
[2025-08-28 16:30:17.329] [info] Group medium size: 1
[2025-08-28 16:30:17.329] [info] Group large size: 0
[2025-08-28 16:30:17.329] [info] Group extra size: 0
[2025-08-28 16:30:17.329] [info] Group overflow size: 0
[2025-08-28 16:30:17.329] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


[2025-08-28 16:30:56.193] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 16:30:56.193] [info] Receptor loaded: 2390 atoms in box
[2025-08-28 16:30:56.194] [info] Ligands loaded. Total count: 1
[2025-08-28 16:30:56.256] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 16:30:56.256] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=0.0044555664 Angstrom, x_hi=30.004456 Angstrom, y_lo=-16.513166 Angstrom, y_hi=13.486834 Angstrom, z_lo=-15.813394 Angstrom, z_hi=14.186606 Angstrom, 
constraint_docking=true, exhaustiveness=512, 
mc_steps=40, opt_steps=-1, refine_steps=5, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-28 16:30:56.256] [info] Group small size: 0
[2025-08-28 16:30:56.256] [info] Group medium size: 1
[2025-08-28 16:30:56.256] [info] Group large size: 0
[2025-08-28 16:30:56.256] [info] Group extra size: 0
[2025-08-28 16:30:56.256] [info] Group overflow size: 0
[2025-08-28 16:30:56.256] [info] @@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:777: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:453: UserWarning: 1 A^3 CRYST1 record, this is usually a placeholder. Unit cell dimensions will be set to None.
  warnings.warn("1 A^3 CRYST1 record,"


[2025-08-28 16:31:26.943] [info] ----------------------- RUN Only Scoring -----------------------
[2025-08-28 16:31:26.943] [info] Receptor loaded: 2393 atoms in box
[2025-08-28 16:31:26.944] [info] Ligands loaded. Total count: 1
[2025-08-28 16:31:27.006] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 16:31:27.006] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=0 Angstrom, x_hi=30 Angstrom, y_lo=-16 Angstrom, y_hi=14 Angstrom, z_lo=-16 Angstrom, z_hi=14 Angstrom, 
constraint_docking=false, exhaustiveness=1, 
mc_steps=0, opt_steps=0, refine_steps=0, 
num_pose=1, rmsd_limit=999 Angstrom, 

[2025-08-28 16:31:27.006] [info] Group small size: 0
[2025-08-28 16:31:27.006] [info] Group medium size: 1
[2025-08-28 16:31:27.006] [info] Group large size: 0
[2025-08-28 16:31:27.006] [info] Group extra size: 0
[2025-08-28 16:31:27.006] [info] Group overflow size: 0
[2025-08-28 16:31:27.006] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@

### Cyclic Peptide

In [14]:
dp = "/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_cyclic_peptide"
fn_pdb = "protein.pdb"
fn_sdf_q = "78_1_5_rand.sdf"
fn_sdf_t = "56_35_2.sdf"
center = [20, -1, 85]

mol_q, mol_t, matrix = perform_align_docking(dp, fn_pdb, fn_sdf_q, fn_sdf_t, center, 512, 0.3)



    ██╗   ██╗██████╗ ██████╗
    ██║   ██║██╔══██╗╚════██╗
    ██║   ██║██║  ██║ █████╔╝
    ██║   ██║██║  ██║██╔═══╝
    ╚██████╔╝██████╔╝███████╗
     ╚═════╝ ╚═════╝ ╚══════╝

    DP Technology Docking Toolkit


{'receptor': 'xx.pdb', 'ligand': 'xx.sdf', 'ligand_batch': None, 'center': [12.0, 25.0, 11.0], 'exhaustiveness': 512, 'randomize': True, 'mc_steps': 40, 'opt_steps': -1, 'refine_steps': 5, 'num_pose': 1, 'rmsd_limit': 1.0, 'energy_range': 3.0, 'seed': 12345, 'use_tor_lib': False, 'n_cpu': None, 'gpu_device_id': 0, 'box_size': [30.0, 30.0, 30.0], 'task': 'screen', 'search_mode': 'free', 'template_docking': True, 'reference_sdf_file_name': '56_35_2.sdf', 'compute_center': True, 'core_atom_mapping_dict_list': None, 'covalent_ligand': False, 'covalent_residue_atom_info_list': None, 'preserve_receptor_hydrogen': False, 'temp_dir_name': '/tmp', 'output_receptor_dms_file_name': 'receptor_parameterized.dms', 'output_docking_pose_sdf_file_name': 'unidock2_out_mcs.sdf'}
Target Cente

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-28 17:48:00.622] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 17:48:00.623] [info] Receptor loaded: 1191 atoms in box
[2025-08-28 17:48:00.624] [info] Ligands loaded. Total count: 1
[2025-08-28 17:48:00.683] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 17:48:00.683] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=4.7995014 Angstrom, x_hi=34.7995 Angstrom, y_lo=-16.11083 Angstrom, y_hi=13.889169 Angstrom, z_lo=69.680115 Angstrom, z_hi=99.680115 Angstrom, 
constraint_docking=true, exhaustiveness=512, 
mc_steps=40, opt_steps=-1, refine_steps=5, 
num_pose=1, rmsd_limit=1 Angstrom, 

[2025-08-28 17:48:00.683] [info] Group small size: 0
[2025-08-28 17:48:00.683] [info] Group medium size: 0
[2025-08-28 17:48:00.683] [info] Group large size: 0
[2025-08-28 17:48:00.683] [info] Group extra size: 0
[2025-08-28 17:48:00.683] [info] Group overflow size: 1
[2025-08-28 17:48:00.683] [info] @@@@@@@@@

/home/lccdp/Softs/miniconda3/envs/ud2pub/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1154: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"


[2025-08-28 17:48:15.068] [info] ----------------------- RUN Only Scoring -----------------------
[2025-08-28 17:48:15.068] [info] Receptor loaded: 1172 atoms in box
[2025-08-28 17:48:15.070] [info] Ligands loaded. Total count: 1
[2025-08-28 17:48:15.133] [info] ----------------------- RUN Screening -----------------------
[2025-08-28 17:48:15.133] [info] DockParam: seed=12345, search_score=0, opt_score=0, 
box: x_lo=5 Angstrom, x_hi=35 Angstrom, y_lo=-16 Angstrom, y_hi=14 Angstrom, z_lo=70 Angstrom, z_hi=100 Angstrom, 
constraint_docking=false, exhaustiveness=1, 
mc_steps=0, opt_steps=0, refine_steps=0, 
num_pose=1, rmsd_limit=999 Angstrom, 

[2025-08-28 17:48:15.133] [info] Group small size: 0
[2025-08-28 17:48:15.133] [info] Group medium size: 0
[2025-08-28 17:48:15.133] [info] Group large size: 0
[2025-08-28 17:48:15.133] [info] Group extra size: 0
[2025-08-28 17:48:15.133] [info] Group overflow size: 1
[2025-08-28 17:48:15.133] [info] @@@@@@@@@@@@@@ Tackling Group: small @@@@@@@@@

# Others

## 查看分子结构图

In [7]:

from IPython.display import SVG
from rdkit.Chem.Draw import rdMolDraw2D
from copy import deepcopy

def show_atom_number(mol, label='atomNote'):
    new_mol = deepcopy(mol)
    for atom in new_mol.GetAtoms():
        atom.SetProp(label, str(atom.GetIdx()))
    return new_mol
def draw_mol(mol, highlight_atoms=[], molSize=(600, 400), kekulize=True):
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0], molSize[1])
    drawer.drawOptions().addAtomIndices = True
    drawer.DrawMolecule(mol, highlightAtoms=highlight_atoms)
    drawer.FinishDrawing()
    svg = drawer.GetDrawingText()
    return SVG(svg)

from IPython.display import HTML

def draw_mols(mols: list, highlight_atoms=None, width=300):
    """显示多个分子（横向排列）"""
    if highlight_atoms is None:
        highlight_atoms = [[] for _ in mols]
    
    if not isinstance(width, list):
        width = [width] * len(mols)
    # 生成独立svg
    list_svg = [draw_mol(mol, highlight_atoms=hl, molSize=(w, 400)).data for mol, hl, w in zip(mols, highlight_atoms, width)]
    
    # 创建横向布局HTML
    html_code = f"""
    <div style="display: flex; gap: 20px;">"""
    html_code += "".join([f"<div>{svg.replace('width:600px', f'width:{width}px')}</div>" for svg in list_svg])
    html_code += "</div>"
    return HTML(html_code)


In [13]:
def read_sdf(fp_sdf):
    m1 = Chem.SDMolSupplier(fp_sdf, removeHs=True)[0]
    m1.RemoveAllConformers()
    return m1

In [ ]:
m1 = read_sdf("HNE_Ref.sdf")
m2 = read_sdf("HNE_2.sdf")
draw_mols([m1, m2], width=600)

## Transfer .MOL file to .SDF File

In [22]:
def transfer_mol_to_sdf(mol_file_path):
    fp_sdf = mol_file_path.replace(".mol", ".sdf")
    mol = Chem.MolFromMolFile(mol_file_path)
    Chem.MolToMolFile(mol, fp_sdf)
    return fp_sdf

transfer_mol_to_sdf("/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/test_all_rings.mol")

'/home/lccdp/Projects/clion/Uni-Dock2/unidock/unidock_engine/examples/align_Bace/test_all_rings.sdf'

# 自动流程筛选测试cases

## Get Cases from Rongfeng HuoShan

In [ ]:
import json
import os
import shutil

dp_res = os.path.abspath("vina_bad_zalign")
fp = "vina_bad.dat"
with open(fp, "r") as f:
    data = f.readlines()
    dict_chembl_id_count = dict()

for d in data:
    try:
        line = d.strip()
        dp = os.path.dirname(line)
        fn = os.path.basename(line)
        chembl_id = os.path.basename(dp)

        fp_json = os.path.join(dp, "1/docking_tree.json")
        with open(fp_json, "r") as f:
            data_tree = json.load(f)
        
        target = fn.split("_3d_dock")[0].strip()
        source = [d["source"] for d in data_tree["edges"] if d["target"] == target][0].strip()
        
        fp_target = os.path.join(dp, f"{target}.sdf")
        fp_target_dock = os.path.join(dp, f"{target}_3d_dock.sdf")
        fp_source = os.path.join(dp, f"{source}.sdf")
        fp_source_dock = os.path.join(dp, f"{source}_3d_dock.sdf")
        fp_pdb = os.path.join(dp, [a for a in os.listdir(dp) if a.endswith(".pdb")][0])
        
        if dict_chembl_id_count.get(chembl_id, 0) == 0:
            dict_chembl_id_count[chembl_id] = 1
        else:
            dict_chembl_id_count[chembl_id] += 1

    
        dp_pair = os.path.join(dp_res, f"{chembl_id}_{dict_chembl_id_count[chembl_id]}")
        os.makedirs(dp_pair, exist_ok=True)

        shutil.copy(fp_target, os.path.join(dp_pair, f"q_{target}.sdf"))
        shutil.copy(fp_target_dock, os.path.join(dp_pair, f"q_{target}_3d_dock.sdf"))
        shutil.copy(fp_source, os.path.join(dp_pair, f"t_{source}.sdf"))
        shutil.copy(fp_source_dock, os.path.join(dp_pair, f"t_{source}_3d_dock.sdf"))
        shutil.copy(fp_pdb, os.path.join(dp_pair, f"{chembl_id}.pdb"))

    except:
        continue